In [2]:
with open('the-verdict.txt','r', encoding='utf-8') as f:
    raw_text = f.read()

print("Lenght of the raw text file is :", len(raw_text))

Lenght of the raw text file is : 20479


In [5]:
#!pip install tiktoken

In [6]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
enc_text = tokenizer.encode(raw_text)
print("The lenght of the encoded token is :", len(enc_text))

The lenght of the encoded token is : 5145


In [7]:
context_size = 6
x = enc_text[:context_size]
y = enc_text[1:context_size+1]
print("X :", x)
print("Y :", y)

X : [40, 367, 2885, 1464, 1807, 3619]
Y : [367, 2885, 1464, 1807, 3619, 402]


In [8]:
for i in range(1, context_size):
    input = enc_text[:i]
    target = enc_text[i]

    print(input, "--->", target)

[40] ---> 367
[40, 367] ---> 2885
[40, 367, 2885] ---> 1464
[40, 367, 2885, 1464] ---> 1807
[40, 367, 2885, 1464, 1807] ---> 3619


In [9]:
enc_sample = enc_text[:20]
for i in range(1, context_size):
    input = enc_sample[:i]
    target = enc_sample[i]

    print(tokenizer.decode(input), "--->", tokenizer.decode([target]))

I --->  H
I H ---> AD
I HAD --->  always
I HAD always --->  thought
I HAD always thought --->  Jack


In [10]:
from torch.utils.data import Dataset, DataLoader
import torch
print(torch.__version__)

class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_len, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special= {"<|endoftext|>"})

        for i in range(0, len(token_ids)-max_len, stride):
            input_chunk = token_ids[i:i + max_len]
            target_chunk = [token_ids[i+1:i + max_len + 1]]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]

2.5.1+cu121


In [11]:
def create_dataloader(txt, shuffle=True, max_lenght=256, stride=1, batch_size=4, drop_last=True , num_workers=0):
    dataset = GPTDataset(txt, tokenizer, max_lenght, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
    return dataloader

In [12]:
dataloader = create_dataloader(raw_text, max_lenght=4, stride=1, batch_size=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[[ 367, 2885, 1464, 1807]]])]
